<h1>Goals</h1>

Find the country that produces the highest percentage of Bottom 50 players. 
<br>Define the correlation between international reputation and market value especially when the reputation is low.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import math

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

········


In [5]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_csv('fifa21_male2.csv')
df

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [7]:
df.describe()

,ID,Age,OVA,BOV,POT,Growth,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY
count,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17067.000000,17125.000000,17125.000000,17067.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17067.000000,17125.000000,17067.000000,17125.000000,17125.000000,17067.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17118.000000,17118.000000,17067.000000,17125.000000,16702.000000,17125.000000,17125.000000,17125.000000,17067.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000,17125.000000
mean,219388.716204,25.272934,66.965022,67.900204,72.489810,5.524788,258.537752,51.621314,47.956672,53.598832,60.507620,45.005742,266.607591,57.846423,49.574676,44.387270,54.324321,60.642803,322.693664,65.445898,65.443036,64.602273,62.924847,64.715591,302.414774,59.711416,65.170212,63.309606,65.308555,49.135708,261.922803,56.997839,47.085582,52.372766,55.443605,50.251445,59.940965,141.469898,47.246949,48.279358,46.099725,77.609693,15.600467,15.484672,15.472058,15.514277,15.738628,1631.256175,361.372146,68.091620,54.967299,58.926540,64.209869,50.266102,64.910715
std,37499.197507,4.942665,6.864329,6.637538,5.769949,5.787539,72.255687,17.875715,19.391547,16.947164,13.960635,17.729483,76.809344,18.047250,18.188344,17.504128,14.856911,15.873647,54.933593,14.657889,14.437377,14.372443,9.037279,14.031563,49.380576,13.417931,11.918063,15.313272,12.627212,19.044862,62.913935,17.021619,20.846705,19.094432,13.730080,15.696363,11.751111,61.595180,20.182548,21.483597,21.185154,81.136603,16.828637,16.143679,15.997328,16.406064,17.130908,260.357024,40.252290,11.147116,13.877605,10.186728,9.832734,16.853801,9.660260
min,2.000000,16.000000,38.000000,42.000000,47.000000,-1.000000,33.000000,6.000000,3.000000,5.000000,7.000000,4.000000,43.000000,5.000000,4.000000,5.000000,9.000000,5.000000,113.000000,12.000000,11.000000,14.000000,24.000000,17.000000,128.000000,12.000000,22.000000,11.000000,16.000000,4.000000,50.000000,9.000000,4.000000,2.000000,10.000000,7.000000,12.000000,17.000000,3.000000,5.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,731.000000,228.000000,25.000000,16.000000,25.000000,28.000000,12.000000,27.000000
25%,204082.000000,21.000000,62.000000,64.000000,69.000000,0.000000,232.000000,41.000000,33.000000,46.000000,56.000000,32.000000,232.000000,53.000000,37.000000,32.000000,45.000000,57.000000,294.000000,58.000000,59.000000,57.000000,57.000000,57.000000,272.000000,50.000000,58.000000,56.000000,58.000000,35.000000,235.000000,45.000000,26.000000,43.000000,47.000000,40.000000,53.000000,84.000000,29.000000,28.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1492.000000,333.000000,62.000000,46.000000,52.000000,59.000000,35.000000,59.000000
50%,228961.000000,25.000000,67.000000,68.000000,72.000000,4.000000,271.000000,56.000000,52.000000,57.000000,63.000000,46.000000,279.000000,63.000000,51.000000,43.000000,57.000000,64.000000,331.000000,68.000000,68.000000,67.000000,63.000000,67.000000,308.000000,61.000000,66.000000,66.000000,67.000000,53.000000,269.000000,60.000000,53.000000,57.000000,57.000000,51.000000,61.000000,158.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1659.000000,362.000000,69.000000,58.000000,60.000000,65.000000,53.000000,66

In [8]:
df.columns = [i.lower().replace(" ", "_") for i in  df.columns]
df.columns

Index(['id', 'name', 'age', 'ova', 'nationality', 'club', 'bov', 'bp',
       'position', 'player_photo',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'gender'],
      dtype='object', length=107)

<h3>Define Dataframe</h3>

In [9]:
#Create dataframe using the columns relevant to the goals
proj_df = df.filter(['name', 'club', 'nationality','ova', 'value', 'ir', 'total_stats'], axis=1)
proj_df

,name,club,nationality,ova,value,ir,total_stats
0,G. Pasquale,Udinese,Italy,69,€625K,2 ★,1929
1,Luis García,KAS Eupen,Spain,71,€600K,1 ★,1906
2,J. Cole,Coventry City,England,71,€1.1M,2 ★,1770
3,D. Yorke,Sunderland,Trinidad &amp; Tobago,68,€0,1 ★,1348
4,Iniesta,Vissel Kobe,Spain,81,€5.5M,4 ★,2014
...,...,...,...,...,...,...,...
17120,A. Medioub,CD Tondela,France,64,€550K,1 ★,1367
17121,W. Rickard,Burnley,Wales,56,€130K,1 ★,1529
17122,C. Barrett,Burnley,England,56,€130K,1 ★,1430
17123,J. Gazibegović,SK Sturm Graz,Bosnia Herzegovina,62,€475K,1 ★,1579


In [10]:
# Provide the number of rows and columns in this dataset

num_rows = proj_df.shape[0] #Provide the number of rows in the dataset
num_cols = proj_df.shape[1] #Provide the number of columns in the dataset

print('There are {} rows in the dataset'.format(num_rows))
print('There are {} columns in the dataset'.format(num_cols))

There are 17125 rows in the dataset
There are 7 columns in the dataset


In [11]:
proj_df.head()

,name,club,nationality,ova,value,ir,total_stats
0,G. Pasquale,Udinese,Italy,69,€625K,2 ★,1929
1,Luis García,KAS Eupen,Spain,71,€600K,1 ★,1906
2,J. Cole,Coventry City,England,71,€1.1M,2 ★,1770
3,D. Yorke,Sunderland,Trinidad &amp; Tobago,68,€0,1 ★,1348
4,Iniesta,Vissel Kobe,Spain,81,€5.5M,4 ★,2014


In [12]:
#Explore data on new dataset
proj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17125 entries, 0 to 17124
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         17125 non-null  object
 1   club         17102 non-null  object
 2   nationality  17125 non-null  object
 3   ova          17125 non-null  int64 
 4   value        17125 non-null  object
 5   ir           17125 non-null  object
 6   total_stats  17125 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 936.6+ KB


In [13]:
#describe numerical columns
proj_df.describe()

,ova,total_stats
count,17125.000000,17125.000000
mean,66.965022,1631.256175
std,6.864329,260.357024
min,38.000000,731.000000
25%,62.000000,1492.000000
50%,67.000000,1659.000000
75%,72.000000,1812.000000
max,93.000000,2316.000000


<h3>Data Cleaning</h3>

In [14]:
#Using the info from the exploration

#Remove star symbols from 'ir' column 
proj_df['ir'] = proj_df['ir'].map(lambda x: x.lstrip('★').rstrip('★'))
proj_df

,name,club,nationality,ova,value,ir,total_stats
0,G. Pasquale,Udinese,Italy,69,€625K,2,1929
1,Luis García,KAS Eupen,Spain,71,€600K,1,1906
2,J. Cole,Coventry City,England,71,€1.1M,2,1770
3,D. Yorke,Sunderland,Trinidad &amp; Tobago,68,€0,1,1348
4,Iniesta,Vissel Kobe,Spain,81,€5.5M,4,2014
...,...,...,...,...,...,...,...
17120,A. Medioub,CD Tondela,France,64,€550K,1,1367
17121,W. Rickard,Burnley,Wales,56,€130K,1,1529
17122,C. Barrett,Burnley,England,56,€130K,1,1430
17123,J. Gazibegović,SK Sturm Graz,Bosnia Herzegovina,62,€475K,1,1579


In [15]:
#Convert 'ir' column to numerical values
type(proj_df['ir'])

pandas.core.series.Series

In [16]:
proj_df['ir'].unique()

array(['2 ', '1 ', '4 ', '3 ', '5 '], dtype=object)

In [17]:
proj_df['ir'] = pd.to_numeric(proj_df['ir'])

In [18]:
proj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17125 entries, 0 to 17124
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         17125 non-null  object
 1   club         17102 non-null  object
 2   nationality  17125 non-null  object
 3   ova          17125 non-null  int64 
 4   value        17125 non-null  object
 5   ir           17125 non-null  int64 
 6   total_stats  17125 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 936.6+ KB


In [20]:
#Remove the Euro symbol from 'value' column
euro = proj_df['value'].map(lambda x: x.lstrip('€').rstrip('€'))
euro

0        625K
1        600K
2        1.1M
3           0
4        5.5M
         ... 
17120    550K
17121    130K
17122    130K
17123    475K
17124    325K
Name: value, Length: 17125, dtype: object


In [22]:
proj_df['value'] = euro
proj_df

,name,club,nationality,ova,value,ir,total_stats
0,G. Pasquale,Udinese,Italy,69,625K,2,1929
1,Luis García,KAS Eupen,Spain,71,600K,1,1906
2,J. Cole,Coventry City,England,71,1.1M,2,1770
3,D. Yorke,Sunderland,Trinidad &amp; Tobago,68,0,1,1348
4,Iniesta,Vissel Kobe,Spain,81,5.5M,4,2014
...,...,...,...,...,...,...,...
17120,A. Medioub,CD Tondela,France,64,550K,1,1367
17121,W. Rickard,Burnley,Wales,56,130K,1,1529
17122,C. Barrett,Burnley,England,56,130K,1,1430
17123,J. Gazibegović,SK Sturm Graz,Bosnia Herzegovina,62,475K,1,1579


In [47]:
dol1 = [float(i.replace('M', ''))*1000000 if 'M' in i else float(i.replace('K', ''))*1000 for i in proj_df['value']]
dol1

TypeError: argument of type 'float' is not iterable

In [49]:
proj_df

,name,club,nationality,ova,value,ir,total_stats
0,G. Pasquale,Udinese,Italy,69,625000.0,2,1929
1,Luis García,KAS Eupen,Spain,71,600000.0,1,1906
2,J. Cole,Coventry City,England,71,1100000.0,2,1770
3,D. Yorke,Sunderland,Trinidad &amp; Tobago,68,0.0,1,1348
4,Iniesta,Vissel Kobe,Spain,81,5500000.0,4,2014
...,...,...,...,...,...,...,...
17120,A. Medioub,CD Tondela,France,64,550000.0,1,1367
17121,W. Rickard,Burnley,Wales,56,130000.0,1,1529
17122,C. Barrett,Burnley,England,56,130000.0,1,1430
17123,J. Gazibegović,SK Sturm Graz,Bosnia Herzegovina,62,475000.0,1,1579


In [26]:
# Identify null values
proj_df.isna().sum()

name            0
club           23
nationality     0
ova             0
value           0
ir              0
total_stats     0
dtype: int64

In [23]:
proj_df.to_csv('fifa_table.csv', index= False)

In [50]:
proj_df.to_csv('fifa_table_sql.csv', index = False)

In [51]:
ls

FIFA_Mini_Project.ipynb  fifa21_male2.csv         fifa_table_sql.csv
FIFA_Project1.ipynb      fifa_table.csv           project_details.md


<h3>Question 1</h3>

<h3>Which country produces the highest percentage of players in the Bottom 50?</h3>

In [35]:
#Which country produces the highest percentage of low ranking players
q1 = df.filter(['nationality','ova', ], axis=1) ##defining dataframe
q1

,nationality,ova
0,Italy,69
1,Spain,71
2,England,71
3,Trinidad &amp; Tobago,68
4,Spain,81
...,...,...
17120,France,64
17121,Wales,56
17122,England,56
17123,Bosnia Herzegovina,62


In [ ]:
worst_players = q1.sort_values(['ova'],descending=[False]).tail(50)

selected = worst_players[['nationality', 'ova']]
selected.tail(50)

In [64]:
type(q1['nationality'])

pandas.core.series.Series

In [55]:
pip install folium

     |████████████████████████████████| 95 kB 3.0 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [56]:
import folium

In [57]:
m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
m

In [62]:
for i in range(0,50(q1)):
   folium.Circle(
      location=[data.iloc[i]['lat'], q1.iloc[i]['lon']],
      popup=data.iloc[i]['nationality'],
      radius=float(data.iloc[i]['ova'])*100,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)

TypeError: 'int' object is not callable